### 원하는 검색건수만큼 검색결과의 각 상세페이지에서 데이터 추출해오기

crawling할 keyword를 입력하고
웹사이트에서 조회되는 검색 결과에서 
원하는 검색건수만큼 상세페이지를 조회하고 각 상세페이지에서 데이터를 추출한다.

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import math

query_txt = input('crawling할 키워드 입력:')
cnt = int(input('crawling할 건수를 입력하세요'))
page_cnt = math.ceil(cnt/10)

ft_name = input('txt파일명과 경로를 지정하세요(예:C:\\study\\python\\notebook\\data\\test.txt): ')
fc_name = input('검색 결과를 저장할 csv 파일경로와 이름을 지정하세요(예:C:\\study\\python\\notebook\\data\\test.csv): ')
fx_name = input('검색 결과를 저장할 xls 파일경로와 이름을 지정하세요(C:\\study\\python\\notebook\\data\\test.xlsx): ')
 
path = "C:/Study/Python/datadown/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://korean.visitkorea.or.kr/")

time.sleep(2)

try:
    driver.find_element_by_xpath('//*[@id="chkForm01"]').click()
except:
    print('코로나 창이 없습니다.')

element = driver.find_element_by_id('inp_search')
element.send_keys(query_txt)
element.send_keys("\n")

time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

print("\n")
print("-"*70)
print("요청하신 검색 건수는 {}건이였고" .format(cnt))
print("총 {}페이지의 게시글 수집을 진행합니다." .format(page_cnt))
print("-"*70)
print("\n")

In [ ]:
no = 1
no2=[]
contents2=[]
tags2=[]

In [ ]:
s_time = time.time()

#끝까지 for loop돌기위해 page_cnt+1
for x in range(1,page_cnt+1):
    print("%s 페이지 내용 수집 시작합니다","-"*40)
    print("\n")
    
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    content_list = soup.find('ul','list_thumType type1')
    
    for i in content_list:
        no2.append(no)
        print('번호',no)
        
        try:
            contents = i.find('div','tit').get_text()
        except AttributeError:
            contents = '내용이 없음'
            contents2.append(contents)
            print('내용:', contents.strip())
        else:
            contents2.append(contents)
            print('내용:', contents.strip())
            
        try:
            tags = i.find('p','tag_type').get_text()
        except AttributeError:
            tags = '태그정보 없음'
            tags2.append(tags)
            print('태그',tags.strip())
        else:
            tags2.append(tags)
            print('태그',tags.strip())
            
        if no == cnt:
            break
        no+=1
        
    time.sleep(2)
    x+=1
    if x == page_cnt+1:
        break
        
    if x == 6:
        driver.find_element_by_link_text("다음").click()
    else:
        driver.find_element_by_link_text("%s" %x).click()

In [ ]:
korea = pd.DataFrame()
korea['번호']=no2
korea['내용']=contents2
korea['태그']=tags2

korea

In [ ]:
f = open(ft_name,'a',encoding='utf-8')
f.write(str(no2))
f.write(str(contents2))
f.write(str(tags2))
f.close()

In [ ]:
korea.to_csv(fc_name, encoding='utf-8', index=False)
korea.to_excel(fx_name, index=False)

In [ ]:
e_time = time.time()
d_time = e_time - s_time

In [ ]:
print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no))
print("총 소요시간은 %s 초 입니다 " %round(d_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ft_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)
 
driver.close( )